In [1]:
import email
import re

from collections import Counter
from dateutil.parser import parse
from pathlib import Path

In [14]:
email_dir = Path(Path.cwd().parent, Path('data/raw/maildir'))

clean_emails = []
for path in email_dir.rglob('*.'):
    if 'all_documents' not in str(path.parent) and 'discussion_threads' not in str(path.parent):
        clean_emails.append(path)


def parse_emails(path):
    with open(path, 'r', encoding='windows-1252') as f:
        parsed_email = email.message_from_file(f)
    return parsed_email


#check to see if an email is part of chain by looking for the '-----Original Message-----' tag
#addtionally count the number of times the tag appears to get the depth of the chain
def get_chain(payload):
    chain_count = 0
    is_chain = False
    check_value = '-----Original Message-----'
    if check_value in payload:
        chain_count = payload.count(check_value)
        is_chain = True
    return chain_count, is_chain


def get_parsed_emails(paths):
    emails = []
    for i, path in enumerate(paths):
        eml = parse_emails(path)
        tms = int(parse(eml['Date']).timestamp())
        chain_count, is_chain = get_chain(eml.get_payload())
        emails.append((i, eml, tms, chain_count, is_chain))
    return emails


parsed_emails = get_parsed_emails(clean_emails)

# lengths = [(text[0], text[1]['message-id'],len(text[1].get_payload())) for text in parsed_emails]
# large_emails = sorted(lengths, key=lambda l: l[2], reverse=True)[:10]
# # large_emails
# l_ids = list(zip(*large_emails))[0]

In [15]:
chain_emails = [email for email in parsed_emails if email[4]]

In [16]:
len(chain_emails)

65075

In [3]:
# for i in l_ids:
#     print(clean_emails[i])

In [4]:
# lengths = [(text[0], text[1]['message-id'], len(preprocess_message(text[1].get_payload()))) for text in parsed_emails]
# large_emails = sorted(lengths, key=lambda l: l[2], reverse=True)[:10]
# large_emails

In [5]:



#check to see if an email is forwarded by looking for 'fw' and 'fwd' tags
def is_forwarded(email):
    is_forwarded = False
    check_values = ['fw:', 'fwd:']
    for value in check_values:
        if value in email.get_payload().lower() or value in email['Subject'].lower():
            is_forwarded = True
    return is_forwarded


#get only the names from the x-headers in the e-mail
def parseXHeaders(header):
    if header is not None:
        return re.sub(r'(<.*?>,|<.*?>)', '|', header)[:-1]
    else:
        return header 


def get_emails_list(dir_path):
    clean_emails = []
    for path in dir_path.rglob('*.'):
        if 'all_documents' not in str(path.parent) and 'discussion_threads' not in str(path.parent):
            clean_emails.append(path)
    return clean_emails


def parse_emails(path):
    with open(path, 'r', encoding='windows-1252') as f:
        parsed_email = email.message_from_file(f)
    return parsed_email


def load_emails(dir_path):
    parsed_emails = []
    clean_emails = get_emails_list(dir_path)
    for i, path in enumerate(clean_emails):
        parsed_emails.append((i,parse_emails(path)))
    return parsed_emails


def preprocess_message(text):
    msg_end_pattern = re.compile('_{4,}.*|\n{3,}|<[^>]*>|-{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE)
    try:
        msg_end_iter = msg_end_pattern.search(text).start()
        # print('end of line:', msg_end_iter)
        message = text[:msg_end_iter]
    except AttributeError: # not a reply
        message = text
    return message


def remove_spaces(string):
    if string is not None:        
        string = re.sub('\s+', ' ', string)
        string = string.split(', ')
    return string


#
# Function: get_or_allocated_uid
# Arguments: name - string of a user email
# Returns: unique integer id
#
def get_or_allocate_uid(name):
     if name not in users:
         users[name] = len(users)
     return users[name]

#
# Function: get_or_allocate_tid
# Arguments: name - string of email subject line
# Returns: unique integer id
#
def get_or_allocate_tid(name):
    parsed_name = re.sub("(RE|Re|FWD|Fwd): ", "", name)
    if parsed_name not in threads:
        threads[parsed_name] = len(threads)
    return threads[parsed_name]

In [6]:
time = [email[1]['Date'] for email in parsed_emails]
subjects = [email[1]['Subject'] for email in parsed_emails]
recipients = [remove_spaces(email[1]['To']) for email in parsed_emails]
senders = [email[1]['From'] for email in parsed_emails]
ccs = [remove_spaces(email[1]['cc']) for email in parsed_emails]
bccs = [remove_spaces(email[1]['bcc']) for email in parsed_emails]
idx = [email[0] for email in parsed_emails]
timestamps = [email[2] for email in parsed_emails]
# body = [email[1].get_payload() for email in parsed_emails]
fwds = [is_forwarded(email[1]) for email in parsed_emails]
chain_count, is_chain = zip(*[is_chain(email[1].get_payload()) for email in parsed_emails])

In [8]:
from statistics import mean

In [12]:
sum(fwds)

39779

In [11]:
mean(chain_count)

0.37717311431586775

In [13]:
sum(is_chain)

65075

In [ ]:
for email in sel_emails[:5]:
    print('Date: ', email[1]['Date'])
    print('Subject: ', email[1]['Subject'])
    print('Sender: ', email[1]['From'])
    print('Receiver: ', email[1]['To'])

In [ ]:
origins = [email[1]['X-Origin'] for email in sel_emails]

In [ ]:
sel_emails[0][1].items()

In [ ]:
Counter(origins).most_common()

In [ ]:
time = [email[1]['Date'] for email in parsed_emails]
subjects = [email[1]['Subject'] for email in parsed_emails]
recipients = [remove_spaces(email[1]['To']) for email in parsed_emails]
senders = [email[1]['From'] for email in parsed_emails]
ccs = [remove_spaces(email[1]['cc']) for email in parsed_emails]
bccs = [remove_spaces(email[1]['bcc']) for email in parsed_emails]
idx = [email[0] for email in parsed_emails]
timestamps = [email[2] for email in parsed_emails]

In [ ]:
message = [preprocess_message(text[1].get_payload()) for text in parsed_emails]

In [ ]:
feeds = []
users = {}
threads = {}
thread_users = {}
user_threads = {}

In [ ]:
parsed_emails[0][1]

In [ ]:
round(parse(time[0]).timestamp())

In [ ]:
Counter(timestamps).most_common()[:10]

In [ ]:
origins = [email[1]['X-Origin'] for email in sel_emails]
Counter(origins).most_common()[:10]

In [ ]:
threads = {}
thread_id = []
for sbj in subject:
    thread_id.append(get_or_allocate_tid(sbj))

In [ ]:
len(thread_id)

In [ ]:
threads

In [ ]:
ts = [i for i in timestamps if i == 993682920]

In [ ]:
parsed_emails[0]

In [ ]:
# ign_dir = sorted(sorted(email_dir.glob('**/discussion_threads')) + sorted(email_dir.glob('**/all_documents')))
# emails = sorted(email_dir.rglob("*."))
# len(emails)
# clean_emails = [path for path in email_dir.rglob('*.') if 'discussion_threads' not in str(path.parent)]
# len(clean_emails)
# len(emails) - len(clean_emails)
# clean_emails = [path for path in email_dir.rglob('*.') if 'all_documents' not in str(path.parent)]
# len(clean_emails)
# len(emails) - len(clean_emails)
# len(parsed_emails)
# parsed_emails[0][1].items()

# parsed_emails[0][1]
# clean_emails[0]
# parsed_emails[0][1]['X-Origin']
# origins = [email[1]['X-Origin'] for email in parsed_emails]
# origins = []
# for email in parsed_emails:
#     origin = email[1]['X-Origin']
#     if origin is not None:
#         origins.append(origin.lower())
#     else:
#         origins.append(origin)
# print(len(set(origins)))
# Counter(origins).most_common()[:10]
# parsed_emails[1][1]
# senders = []
# for email in parsed_emails:
#     sender = email[1]['From']
#     senders.append(sender)
# Counter(senders).most_common()[:10]
# subjects = []
# for email in parsed_emails:
#     subject = email[1]['Subject']
#     subjects.append(subject)
# Counter(subjects).most_common()[:20]

# Precompiled patterns for performance
#
# time_pattern = re.compile("Date: (?P<data>[A-Z][a-z]+\, \d{1,2} [A-Z][a-z]+ \d{4} \d{2}\:\d{2}\:\d{2} \-\d{4} \([A-Z]{3}\))")
# subject_pattern = re.compile("Subject: (?P<data>.*)")
# sender_pattern = re.compile("From: (?P<data>.*)")
# recipient_pattern = re.compile("To: (?P<data>.*)")
# cc_pattern = re.compile("cc: (?P<data>.*)")
# bcc_pattern = re.compile("bcc: (?P<data>.*)")
# msg_start_pattern = re.compile("\n\n", re.MULTILINE)
# msg_end_pattern = re.compile("\n+.*\n\d+/\d+/\d+ \d+:\d+ [AP]M", re.MULTILINE)
# re.compile('-{4,}.*|-{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE)

# text = sample_raw_email.replace("\r", "")
# time = time_pattern.search(text).group("data").replace("\n", "")
# print(time)
# print(sample_email['Date'])
# subject = subject_pattern.search(text).group("data").replace("\n", "")
# print(subject)
# print(sample_email['Subject'])
# sender = sender_pattern.search(text).group("data").replace("\n", "")
# print(sender)
# print(sample_email['From'])
# recipient = recipient_pattern.search(text).group("data").split(", ")
# print(recipient)
# print(sample_email['To'].split(','))

# # cc = cc_pattern.search(text).group("data").split(", ")
# # bcc = bcc_pattern.search(text).group("data").split(", ")

# msg_start_iter = msg_start_pattern.search(text).end()
# try:
#     msg_end_iter = msg_end_pattern.search(text).start()
#     message = text[msg_start_iter:msg_end_iter]
# except AttributeError: # not a reply
#     message = text[msg_start_iter:]
    
# message = re.sub("[\n\r]", " ", message)
# print(message)
# message = re.sub("  +", " ", message)
# print(message)

# msg_end_pattern = re.compile('-{4,}.*|[-_]{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE)
# # re.compile('-{4,}.*|-{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE)
# text2 = raw_emails[258598][1]
# # print(text2)
# msg_start_iter = msg_start_pattern.search(text2).end()
# try:
#     msg_end_iter = msg_end_pattern.search(text2).start()
#     message = text2[msg_start_iter:msg_end_iter]
# except AttributeError: # not a reply
#     message = text2[msg_start_iter:3000]

# print(parsed_emails[249493][1].get_payload())

# re.compile('_{4,}.*|\n{3,}|<[^>]*>|-{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE).search(parsed_emails[large_emails[0][0]][1].get_payload()).start()
# re.compile('_{4,}.*', re.MULTILINE).search(parsed_emails[large_emails[6][0]][1].get_payload()).start()
# re.compile('_{4,}.*|\n{3,}|<[^>]*>|-{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE).search(parsed_emails[large_emails[6][0]][1].get_payload()).start()

# time_pattern = re.compile("Date: (?P<data>[A-Z][a-z]+\, \d{1,2} [A-Z][a-z]+ \d{4} \d{2}\:\d{2}\:\d{2} \-\d{4} \([A-Z]{3}\))")
# subject_pattern = re.compile("Subject: (?P<data>.*)")
# sender_pattern = re.compile("From: (?P<data>.*)")
# recipient_pattern = re.compile("To: (?P<data>.*)")
# cc_pattern = re.compile("cc: (?P<data>.*)")
# bcc_pattern = re.compile("bcc: (?P<data>.*)")
# msg_start_pattern = re.compile("\n\n", re.MULTILINE)
# msg_end_pattern = re.compile("\n+.*\n\d+/\d+/\d+ \d+:\d+ [AP]M", re.MULTILINE)
#
# Function: get_or_allocated_uid
# Arguments: name - string of a user email
# Returns: unique integer id
#
# def get_or_allocate_uid(name):
#      if name not in users:
#          users[name] = len(users)
#      return users[name]

#
# Function: get_or_allocate_tid
# Arguments: name - string of email subject line
# Returns: unique integer id
#
# def get_or_allocate_tid(name):
#     parsed_name = re.sub("(RE|Re|FWD|Fwd): ", "", name)
#     if parsed_name not in threads:
#         threads[parsed_name] = len(threads)
#     return threads[parsed_name]

# sender_id = get_or_allocate_uid(sender)
# recipient_id = [get_or_allocate_uid(u.replace("\n", "")) for u in recipient if u!=""]
# cc_ids = [get_or_allocate_uid(u.replace("\n", "")) for u in cc if u!=""]
# bcc_ids = [get_or_allocate_uid(u.replace("\n", "")) for u in bcc if u!=""]
# thread_id = get_or_allocate_tid(subject)

In [ ]:
def parse_email(pathname, orig=True):
    if path.isdir(pathname):
        print(pathname)
        emails = []
        for child in listdir(pathname):
            # only parse visible files
            if child[0] != ".":
                parse_email(path.join(pathname, child), False)
    else:
        print("file %s" % pathname)
        with open(pathname) as TextFile:
            text = TextFile.read().replace("\r", "")
            try:
                time = time_pattern.search(text).group("data").replace("\n", "")
                subject = subject_pattern.search(text).group("data").replace("\n", "")

                sender = sender_pattern.search(text).group("data").replace("\n", "")

                recipient = recipient_pattern.search(text).group("data").split(", ")
                cc = cc_pattern.search(text).group("data").split(", ")
                bcc = bcc_pattern.search(text).group("data").split(", ")
                msg_start_iter = msg_start_pattern.search(text).end()
                try:
                    msg_end_iter = msg_end_pattern.search(text).start()
                    message = text[msg_start_iter:msg_end_iter]
                except AttributeError: # not a reply
                    message = text[msg_start_iter:]
                message = re.sub("[\n\r]", " ", message)
                message = re.sub("  +", " ", message)
            except AttributeError:
                logging.error("Failed to parse %s" % pathname) 
                return None
            # get user and thread ids
            sender_id = get_or_allocate_uid(sender)
            recipient_id = [get_or_allocate_uid(u.replace("\n", "")) for u in recipient if u!=""]
            cc_ids = [get_or_allocate_uid(u.replace("\n", "")) for u in cc if u!=""]
            bcc_ids = [get_or_allocate_uid(u.replace("\n", "")) for u in bcc if u!=""]
            thread_id = get_or_allocate_tid(subject)
        if thread_id not in thread_users:
            thread_users[thread_id] = set()
        # maintain list of users involved in thread
        users_involved = []
        users_involved.append(sender_id)
        users_involved.extend(recipient_id)
        users_involved.extend(cc_ids)
        users_involved.extend(bcc_ids)
        thread_users[thread_id] |= set(users_involved)
        # maintain list of threads where user is involved
        for user in set(users_involved):
            if user not in user_threads:
                user_threads[user] = set()
            user_threads[user].add(thread_id)
 
        entry =  {"time": time, "thread": thread_id, "sender": sender_id, "recipient": recipient_id, "cc": cc_ids, "bcc": bcc_ids, "message": message}
        feeds.append(entry)
    if orig:
        try:
            with open('messages.json', 'w') as f:
                json.dump(feeds, f)
            with open('users.json', 'w') as f:
                json.dump(users, f)
            with open('threads.json', 'w') as f:
                json.dump(threads, f)
            with open('thread-users.json', 'w') as f:
                for thread in thread_users:
                    thread_users[thread] = list(thread_users[thread])
                json.dump(thread_users, f)
            with open('user-threads.json', 'w') as f:
                for user in user_threads:
                    user_threads[user] = list(user_threads[user])
                json.dump(user_threads, f)
        except IOError:
            print("Unable to write to output files, aborting")
            exit(1)

# Doc2Vec tests

In [1]:
import math
import numpy as np
import pickle as pkl
import random

import matplotlib
import matplotlib.pyplot as plt
# import nltk

from collections import Counter
from pathlib import Path

from hdbscan import HDBSCAN
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import TruncatedSVD 
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics
from sklearn.metrics import pairwise_distances
# nltk.download('punkt')

In [2]:
data_dir = Path(Path.cwd().parent, 'data/interim')
models_dir = Path(Path.cwd().parent, 'src/models')

In [3]:
with open(Path(data_dir, 'message.pkl'), 'rb') as handle:
    messages = pkl.load(handle)

In [5]:
len(messages)

330689

In [24]:
filtered_3 = [msg for i, msg, t in messages if not str.isspace(msg) or msg != '']

In [27]:
def load_emails(size=int(1e4), seed=42):
    with open(Path(data_dir, 'message.pkl'), 'rb') as handle:
          messages = pkl.load(handle)
    
    filtered_emails = [(i, msg) for i, msg, t in messages if not str.isspace(msg) or msg != '']
    random.seed(seed)
    emails = random.sample(filtered_emails, k=size)
    return emails

In [28]:
emails = load_emails()

In [29]:
# tokenized_documents = []

# for i in range(len(emails)):
#   tokenized_documents.append(nltk.word_tokenize(emails[i]))

# tkn_documents = [nltk.word_tokenize]

tokenized_docs = [simple_preprocess(emails[i][1]) for i in range(len(emails))]
corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(tokenized_docs)]


In [30]:
vector_size = 300
window_size = 15
min_count = 1
train_epoch = 20
alpha = 0.25
min_alpha = 1e-5
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                epochs=train_epoch,
                dm=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [31]:
ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

counter = Counter(ranks)
print([(i, c, c/len(ranks)*100) for i, c in list(counter.most_common()[:10])])

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(corpus[sims[index][0]].words)))

doc_id = random.randint(0, len(corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(corpus[sim_id[0]].words)))

[(0, 864, 8.64), (1, 156, 1.5599999999999998), (2, 84, 0.84), (3, 66, 0.66), (4, 45, 0.44999999999999996), (5, 38, 0.38), (8, 34, 0.33999999999999997), (7, 32, 0.32), (10, 29, 0.29), (6, 27, 0.27)]
Document (9999): «thanks for update pls keep sending info»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dbow,d300,n5,s0.001,t3>:

MOST (5972, 0.6624950766563416): «please let me know if you have any questions thanks fran»

SECOND-MOST (8444, 0.6498376727104187): «thanks tim»

MEDIAN (6174, 0.26545777916908264): «please review the home addresss and phone number shown on this email do not respond if the information is correct however if incorrect then reply to this email with the correct information so you hr records can be updated accordingly your home information is important in case the company needs to forward critical materials to you or contact you regarding important company information»

LEAST (7326, -0.2550029456615448): «forwarded by lisa yoho na enron on am»

Train Document (7486): «h

In [ ]:
vector_size = 100
window_size = 10
min_count = 1
train_epoch = 20
alpha = 0.25
min_alpha = 1e-4
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                epochs=train_epoch,
                dm=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])
counter = Counter(ranks)
print([(i, c, c/len(ranks)*100) for i, c in list(counter.most_common()[:10])])

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(corpus[sims[index][0]].words)))

doc_id = random.randint(0, len(corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(corpus[sim_id[0]].words)))

In [ ]:
vector_size = 300
window_size = 15
min_count = 1
train_epoch = 50
alpha = 0.25
min_alpha = 1e-4
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                epochs=train_epoch,
                dm=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])
counter = Counter(ranks)
print([(i, c, c/len(ranks)*100) for i, c in list(counter.most_common()[:10])])

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(corpus[sims[index][0]].words)))

doc_id = random.randint(0, len(corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(corpus[sim_id[0]].words)))

In [ ]:
vector_size = 300
window_size = 15
min_count = 1
train_epoch = 20
alpha = 0.25
min_alpha = 1e-10
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                epochs=train_epoch,
                dm=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])
counter = Counter(ranks)
print([(i, c, c/len(ranks)*100) for i, c in list(counter.most_common()[:10])])

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(corpus[sims[index][0]].words)))

doc_id = random.randint(0, len(corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(corpus[sim_id[0]].words)))

In [ ]:
vector_size = 300
window_size = 20
min_count = 1
train_epoch = 20
alpha = 0.25
min_alpha = 1e-5
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                epochs=train_epoch,
                dm=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])
counter = Counter(ranks)
print([(i, c, c/len(ranks)*100) for i, c in list(counter.most_common()[:10])])

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(corpus[sims[index][0]].words)))

doc_id = random.randint(0, len(corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(corpus[sim_id[0]].words)))

In [ ]:
vector_size = 300
window_size = 15
min_count = 2
train_epoch = 20
alpha = 0.25
min_alpha = 1e-5
model = Doc2Vec(vector_size=vector_size,
                window=window_size,
                alpha=alpha, 
                min_alpha=min_alpha,
                min_count=min_count,
                epochs=train_epoch,
                dm=0)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)

ranks = []
second_ranks = []
for doc_id in range(len(corpus)):
    inferred_vector = model.infer_vector(corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])
counter = Counter(ranks)
print([(i, c, c/len(ranks)*100) for i, c in list(counter.most_common()[:10])])

print('Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(corpus[sims[index][0]].words)))

doc_id = random.randint(0, len(corpus) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(corpus[sim_id[0]].words)))